## 3.6.2 Airline Arrivals
In this notebook we are going to use airline data to try and predict wether or not a flight will be over 30 minutes late

In [111]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [112]:
df = pd.read_csv(r'C:\Users\jmfra\OneDrive\Documents\Thinkful Data Science Files\3.6.2 data\2008.csv')

In [113]:
#lets take a look
pd.set_option('display.max_columns', 30)
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [114]:
df.describe()

C:\Users\jmfra\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.014612e+06,1.048575e+06,1.011963e+06,1.048575e+06,1.048575e+06,1.011963e+06,1.048400e+06,1.011963e+06,1.011963e+06,1.014612e+06,1.048575e+06,1.011963e+06,1.014612e+06,1.048575e+06,1.048575e+06,270096.000000,270096.000000,270096.000000,270096.000000,270096.000000
mean,2008.0,1.422297e+00,1.572409e+01,3.895182e+00,1.343447e+03,1.331361e+03,1.489051e+03,1.497376e+03,2.359956e+03,1.244470e+02,1.249264e+02,1.008480e+02,1.141087e+01,1.245653e+01,6.881744e+02,6.825616e+00,1.678146e+01,3.238967e-02,2.526286e-03,15.708015,3.254199,15.878591,0.083955,21.974676
std,0.0,4.939256e-01,8.728132e+00,1.938000e+00,4.768043e+02,4.625290e+02,5.027085e+02,4.791596e+02,1.989882e+03,6.770997e+01,6.710413e+01,6.521326e+01,4.093525e+01,3.763313e+01,5.361000e+02,4.890826e+00,1.116645e+01,1.770328e-01,5.019866e-02,39.385498,20.354545,31.174628,1.728502,39.753498
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,-9.000000e+00,0.000000e+00,-9.100000e+01,-9.200000e+01,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.0,1.000000e+00,8.000000e+00,2.000000e+00,NaN,9.300000e+02,NaN,1.119000e+03,6.690000e+02,NaN,NaN,NaN,NaN,NaN,3.130000e+02,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,2008.0,1.000000e+00,1.600000e+01,4.000000e+00,NaN,1.325000e+03,NaN,1.520000e+03,1.660000e+03,NaN,NaN,NaN,NaN,NaN,5.430000e+02,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,2008.0,2.000000e+00,2.300000e+01,5.000000e+00,NaN,1.720000e+03,NaN,1.908000e+03,3.818000e+03,NaN,NaN,NaN,NaN,NaN,8.990000e+02,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
max,2008.0,2.000000e+00,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.202000e+03,6.930000e+02,1.435000e+03,6.450000e+02,2.461000e+03,2.457000e+03,4.962000e+03,2.130000e+02,3.830000e+02,1.000000e+00,1.000000e+00,1455.000000,1049.000000,1357.000000,149.000000,1143.000000


In [115]:
df[df.Cancelled == 1].head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
178,2008,1,3,4,NaN,700,NaN,830,WN,126,NaN,NaN,90.0,NaN,NaN,NaN,LAS,OAK,407,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
373,2008,1,3,4,NaN,1100,NaN,1215,WN,1146,NaN,NaN,75.0,NaN,NaN,NaN,LAX,OAK,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
399,2008,1,3,4,NaN,905,NaN,1025,WN,469,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
401,2008,1,3,4,NaN,1620,NaN,1740,WN,618,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,C,0,NaN,NaN,NaN,NaN,NaN
415,2008,1,3,4,NaN,1930,NaN,2035,WN,2528,NaN,NaN,65.0,NaN,NaN,NaN,LAX,SJC,308,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN


In [116]:
#ok so there are a lot of NaN values that we have to take a look at. from some exploration i noticed that when cancelled=1, there
#is almost no data shown below. I am going to assume this means when the flight is cancelled, they left all data based on the
#flight out. that makes sense but this data is useless to us so lets get rid of it and columns associated with it 
df = df[df.Cancelled != 1]
df = df.drop(['CancellationCode','Cancelled'], 1)

In [117]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,2.0,0.0,0.0,0.0,32.0


In [118]:
df.describe()

C:\Users\jmfra\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1014612.0,1.014612e+06,1.014612e+06,1.014612e+06,1.014612e+06,1.014612e+06,1.011963e+06,1.014612e+06,1.014612e+06,1.011963e+06,1.014459e+06,1.011963e+06,1.011963e+06,1.014612e+06,1.014612e+06,1.011963e+06,1.014612e+06,1.014612e+06,270096.000000,270096.000000,270096.000000,270096.000000,270096.000000
mean,2008.0,1.420018e+00,1.574707e+01,3.903073e+00,1.343447e+03,1.330439e+03,1.489051e+03,1.496685e+03,2.326567e+03,1.244470e+02,1.254799e+02,1.008480e+02,1.141087e+01,1.245653e+01,6.936101e+02,6.825616e+00,1.678146e+01,2.610850e-03,15.708015,3.254199,15.878591,0.083955,21.974676
std,0.0,4.935616e-01,8.713637e+00,1.937608e+00,4.768043e+02,4.626174e+02,5.027085e+02,4.793916e+02,1.974020e+03,6.770997e+01,6.742616e+01,6.521326e+01,4.093525e+01,3.763313e+01,5.386769e+02,4.890826e+00,1.116645e+01,5.102976e-02,39.385498,20.354545,31.174628,1.728502,39.753498
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,-9.000000e+00,0.000000e+00,-9.100000e+01,-9.200000e+01,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.0,1.000000e+00,8.000000e+00,2.000000e+00,9.350000e+02,9.300000e+02,NaN,1.118000e+03,6.570000e+02,NaN,NaN,NaN,NaN,-4.000000e+00,3.170000e+02,NaN,1.000000e+01,0.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,2008.0,1.000000e+00,1.600000e+01,4.000000e+00,1.334000e+03,1.325000e+03,NaN,1.518000e+03,1.630000e+03,NaN,NaN,NaN,NaN,0.000000e+00,5.450000e+02,NaN,1.400000e+01,0.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,2008.0,2.000000e+00,2.300000e+01,5.000000e+00,1.733000e+03,1.717000e+03,NaN,1.907000e+03,3.770000e+03,NaN,NaN,NaN,NaN,1.200000e+01,9.090000e+02,NaN,2.000000e+01,0.000000e+00,NaN,NaN,NaN,NaN,NaN
max,2008.0,2.000000e+00,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.202000e+03,6.930000e+02,6.600000e+02,6.450000e+02,2.461000e+03,2.457000e+03,4.962000e+03,2.130000e+02,3.830000e+02,1.000000e+00,1455.000000,1049.000000,1357.000000,149.000000,1143.000000


In [119]:
#that did not get rid of the null values so lets take a look at them
df.isnull().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                2649
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   0
ActualElapsedTime      2649
CRSElapsedTime          153
AirTime                2649
ArrDelay               2649
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 2649
TaxiOut                   0
Diverted                  0
CarrierDelay         744516
WeatherDelay         744516
NASDelay             744516
SecurityDelay        744516
LateAircraftDelay    744516
dtype: int64

In [120]:
#actual arrival time, actual elapsed time, airtime, arrival delay, and taxi in all have an identical number of nulls, this 
#implies that there might be an issue where some flights were cancelled but not marked as so. lets remove one column of nulls and
#see if that fixes them all 
df = df.dropna(subset=['ArrTime'])
df.isnull().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                   0
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   0
ActualElapsedTime         0
CRSElapsedTime            0
AirTime                   0
ArrDelay                  0
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                    0
TaxiOut                   0
Diverted                  0
CarrierDelay         741867
WeatherDelay         741867
NASDelay             741867
SecurityDelay        741867
LateAircraftDelay    741867
dtype: int64

In [121]:
#Now we have 740 thousand nulls in the last 5 categories. This is over 3/4 of the data. Now we can assume that a null supposed to 
#be 0 minutes or we could drop these columns in our analysis (because we cant drop the rows that include them, that would be way
#too many). The issue here is that since all the categories have an identical number of nulls, it is likely that for most flights
#the type of delay was simply just not recorded. you might have a flight with 4 hour delay that has no specific type and a flight 
#delayed 29 minutes with a mixture of all the delays. Also if these columns do exist and total more than 30, it would be a 
#direct identifier and we wouldn't actually be predicting lateness. for these reason I am going to delete the columns.
df = df.iloc[:, 0:22]

In [122]:
df.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Diverted
count,1011963.0,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1.011963e+06,1011963.0
mean,2008.0,1.419811e+00,1.574887e+01,3.903447e+00,1.343556e+03,1.330616e+03,1.489051e+03,1.496702e+03,2.326004e+03,1.244470e+02,1.254388e+02,1.008480e+02,1.141087e+01,1.240269e+01,6.932789e+02,6.825616e+00,1.677337e+01,0.0
std,0.0,4.935280e-01,8.712593e+00,1.937226e+00,4.767299e+02,4.625653e+02,5.027085e+02,4.792504e+02,1.974017e+03,6.770997e+01,6.739783e+01,6.521326e+01,4.093525e+01,3.751126e+01,5.385217e+02,4.890826e+00,1.115038e+01,0.0
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,1.800000e+01,0.000000e+00,-9.100000e+01,-9.200000e+01,2.400000e+01,0.000000e+00,0.000000e+00,0.0
25%,2008.0,1.000000e+00,8.000000e+00,2.000000e+00,9.350000e+02,9.300000e+02,1.114000e+03,1.119000e+03,6.560000e+02,7.600000e+01,7.800000e+01,5.500000e+01,-9.000000e+00,-4.000000e+00,3.170000e+02,4.000000e+00,1.000000e+01,0.0
50%,2008.0,1.000000e+00,1.600000e+01,4.000000e+00,1.334000e+03,1.325000e+03,1.518000e+03,1.518000e+03,1.630000e+03,1.070000e+02,1.080000e+02,8.300000e+01,0.000000e+00,0.000000e+00,5.450000e+02,6.000000e+00,1.400000e+01,0.0
75%,2008.0,2.000000e+00,2.300000e+01,5.000000e+00,1.733000e+03,1.718000e+03,1.912000e+03,1.907000e+03,3.770000e+03,1.530000e+02,1.540000e+02,1.280000e+02,1.600000e+01,1.200000e+01,9.080000e+02,8.000000e+00,2.000000e+01,0.0
max,2008.0,2.000000e+00,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.359000e+03,9.202000e+03,6.930000e+02,6.600000e+02,6.450000e+02,2.461000e+03,2.457000e+03,4.962000e+03,2.130000e+02,3.830000e+02,0.0


In [123]:
#now that we got rid of all the bad values, theres a few things to notice. There is only one year, 2 months and none of the 
#flights remaining were diverted so we can delete all those columns. We will wait to drop Month for now because we use it later
df = df.drop(['Year', 'Diverted'], axis=1)

In [124]:
#We cant use actual arrival time in conjuction with scheduled because that is the definition of what we are trying to predict. We
#will remove it later however as we might need the column.
#day of the month and day of the week are categorical variables represented as numbers. We need to create dummies based on them so
#the models know they are categories. Since actual month actually matters (january 1st and february 1st are completely
#different days) we are going to decide not to drop it like we wanted to before and instead create numbers like 1.21 is the 
#twenty first day of the first month and then make it categorical
df['DayAndMonth'] = df['Month'] + .1 * df['DayofMonth']
df = pd.concat([df, pd.get_dummies(df['DayAndMonth'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['DayOfWeek'])], axis=1)
#Now drop the continuous variables used to create these columns
df = df.drop(['Month', 'DayofMonth', 'DayOfWeek', 'DayAndMonth'], axis=1)

In [125]:
#scheduled departure time and actual are both important but redudant, and we have a departure delay feature already 
#same with Arrival delay 
df = df.drop(['DepTime', 'ArrTime'], axis=1)

In [126]:
#flightnumber and TailNum are arbitrary identifiers with elements of other columns so we can delete them
df = df.drop(['FlightNum', 'TailNum'], axis=1)

In [127]:
#actual elapsed time and scheduled elapsed are redudant but both important. lets use one category and their difference as another
df['DepTimeDiff'] = df['CRSElapsedTime'] - df['ActualElapsedTime'] 
df = df.drop('ActualElapsedTime', axis=1)

In [128]:
#airtime is just some percentage of elapsed time
df = df.drop('AirTime', axis=1)

In [129]:
#now that we looked at all the continuous variables, lets look at the categorical ones
df.select_dtypes(exclude=['floating', 'int64'])

,UniqueCarrier,Origin,Dest
0,WN,IAD,TPA
1,WN,IAD,TPA
2,WN,IND,BWI
3,WN,IND,BWI
4,WN,IND,BWI
5,WN,IND,JAX
6,WN,IND,LAS
7,WN,IND,LAS
8,WN,IND,MCI
9,WN,IND,MCI


In [130]:
#I want to figure out how many of each of these categories exist and their counts 
print(df['UniqueCarrier'].groupby(df['UniqueCarrier']).count().size)
df['UniqueCarrier'].groupby(df['UniqueCarrier']).count()

20


UniqueCarrier
9E     36595
AA     50640
AQ      4000
AS     12289
B6     16207
CO     24946
DL     55994
EV     43351
F9     14754
FL     40220
HA      9017
MQ     77945
NW     59064
OH     34082
OO     90141
UA     71663
US     74538
WN    192409
XE     64963
YV     39145
Name: UniqueCarrier, dtype: int64

In [131]:
print(df['Origin'].groupby(df['Origin']).count().size)
df['Origin'].groupby(df['Origin']).count().describe()

286


count      286.000000
mean      3538.332168
std       7243.654721
min          9.000000
25%        248.250000
50%        709.000000
75%       2608.250000
max      57962.000000
Name: Origin, dtype: float64

In [132]:
print(df['Dest'].groupby(df['Dest']).count().size)
df['Dest'].groupby(df['Dest']).count().describe()

286


count      286.000000
mean      3538.332168
std       7228.060615
min          9.000000
25%        251.250000
50%        712.500000
75%       2623.000000
max      57802.000000
Name: Dest, dtype: float64

In [133]:
#The unique carrier seems like an easy enough variable to turn into dummies but
#the destination and origin have too many unique entries and i know from experience
#will completely freeze my computer. I can figure our where each one is and
#group them into regions but for now I am going to drop them and see how good
#the model is. I will drop them later just in case we end up using them.
df = pd.concat([df, pd.get_dummies(df['UniqueCarrier'])], axis=1)
df = df.drop('UniqueCarrier', axis=1)

In [134]:
df.head()

,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,1.1,1.2,1.3,1.4,1.5,...,CO,DL,EV,F9,FL,HA,MQ,NW,OH,OO,UA,US,WN,XE,YV
0,1955,2225,150.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,735,1000,145.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,620,750,90.0,14.0,8.0,IND,BWI,515,3.0,17.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,930,1100,90.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1755,1925,90.0,34.0,34.0,IND,BWI,515,3.0,10.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [135]:
df['Late']  = np.where(df['ArrDelay'] >= 30,1,0)
df = df.drop('ArrDelay', axis=1)

In [136]:
df['Late'].groupby(df['Late']).count()

Late
0    845858
1    166105
Name: Late, dtype: int64

In [137]:
%%time
#This 80, 20 split might be an issue and we might want to resample with even numbers but lets start off with the whole set and 
#see what numbers we get
lr = LogisticRegression()
y = df['Late']
X = df.drop(['Late', 'Origin', 'Dest'], axis = 1)
# Fit the model.
fit = lr.fit(X, y)
pred_y_sklearn = lr.predict(X)
crosstab = pd.crosstab(pred_y_sklearn, y)
print('actual on top, prediction on left')
print(crosstab)

print('accurately predicted on time percentage')
print(crosstab.iloc[0,0]/(crosstab.iloc[0,0]+crosstab.iloc[1,0]))

print('accurately predicted late percentage')
print(crosstab.iloc[1,1]/(crosstab.iloc[1,1]+crosstab.iloc[0,1]))

print('Percentage accuracy')
print(lr.score(X, y))

actual on top, prediction on left
Late        0       1
row_0                
0      841497    4371
1        4361  161734
accurately predicted on time percentage
0.994844288285
accurately predicted late percentage
0.973685319527
Percentage accuracy
0.991371226023
Wall time: 23.2 s


In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [140]:
# Fit the model.
fit = lr.fit(X_train, y_train)
pred_y_sklearn = lr.predict(X_test)
crosstab = pd.crosstab(pred_y_sklearn, y_test)
print('actual on top, prediction on left')
print(crosstab)

print('accurately predicted on time percentage')
print(crosstab.iloc[0,0]/(crosstab.iloc[0,0]+crosstab.iloc[1,0]))

print('accurately predicted late percentage')
print(crosstab.iloc[1,1]/(crosstab.iloc[1,1]+crosstab.iloc[0,1]))

print('Percentage accuracy')
print(lr.score(X_test, y_test))

actual on top, prediction on left
Late        0      1
row_0               
0      277582   1304
1        1305  53757
accurately predicted on time percentage
0.995320685439
accurately predicted late percentage
0.976317175496
Percentage accuracy
0.992187406423


These results are incredibly good. We are predicting yes or no at over a 99%
accuracy even without information about which airport the flight is going to and from.